In [1]:
from pandas import DataFrame, read_csv
from torch.utils.data import random_split
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import torch

In [2]:
# create labels csv

types = os.listdir("vehicle_classification")
print(types)

conversion = {
    'Bicycle': 0,
    'Bus': 1,
    'Car': 2,
    'Motorcycle': 3,
    'NonVehicles': 4,
    'Taxi': 5,
    'Truck': 6,
    'Van': 7
}
labels = {}
for t in types:
    for img in os.listdir(f"vehicle_classification/{t}"):
        labels[f"{t}\\{img}"] = conversion[t]
df = DataFrame.from_dict(labels, orient='index', columns=["type"])

df.to_csv("labels.csv")

['Bicycle', 'Bus', 'Car', 'Motorcycle', 'NonVehicles', 'Taxi', 'Truck', 'Van']


In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image.type_as(torch.FloatTensor()), label

In [4]:
data = CustomImageDataset("labels.csv", "vehicle_classification", transform=None)
train_data, test_data = random_split(data, [0.8, 0.2])

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


feature_batch, label_batch = next(iter(train_dataloader))
print(f"Shape of features batch [N, C, H, W]: {feature_batch.shape}")
print(f"Length of label batch: {len(label_batch)}")
print(f"Type of X: {feature_batch.dtype}")
print(f"Type of y: {label_batch.dtype}")

Shape of features batch [N, C, H, W]: torch.Size([64, 3, 64, 64])
Length of label batch: 64
Type of X: torch.float32
Type of y: torch.int64


In [6]:
# use gpu/cpu
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.network = nn.Sequential (
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            
            nn.Flatten(),
            
            nn.Linear(in_features=12800, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),

            nn.ReLU(),
            nn.Linear(in_features=64, out_features=8)
        )
        
        

    def forward(self, x):
        out = self.network(x)
        # print (out.shape)
        return self.network(x)     

model = CNN().to(device)
print(model)

Using cuda device
CNN(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Linear(in_features=12800, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=8, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader: DataLoader, model: nn.Module, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (feature, label) in enumerate(dataloader):
        feature, label = feature.to(device), label.to(device)
        pred = model(feature)
        loss = loss_fn(pred, label)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(feature)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return (correct, test_loss)

In [30]:
from time import time
epochs = 5
with open("results.csv", "w") as f:
    f.write("epoch, loss, accuracy, time taken\n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    t0 = time()
    train(train_dataloader, model, loss_fn, optimizer)
    acc, loss = test(test_dataloader, model, loss_fn)
    t1 = time()
    with open("results.csv", "a") as f:
        f.write(f"{t+1}, {loss}, {acc}, {t1-t0}\n")
print("Done!")

Epoch 1
-------------------------------
loss: 0.609913  [   64/21103]
loss: 0.380389  [ 6464/21103]
loss: 0.563296  [12864/21103]
loss: 0.665378  [19264/21103]
Test Error: 
 Accuracy: 73.1%, Avg loss: 0.756520 

Epoch 2
-------------------------------
loss: 0.578369  [   64/21103]
loss: 0.337718  [ 6464/21103]
loss: 0.529570  [12864/21103]
loss: 0.631546  [19264/21103]
Test Error: 
 Accuracy: 71.3%, Avg loss: 0.801114 

Epoch 3
-------------------------------
loss: 0.581722  [   64/21103]
loss: 0.308981  [ 6464/21103]
loss: 0.496208  [12864/21103]
loss: 0.609135  [19264/21103]
Test Error: 
 Accuracy: 72.2%, Avg loss: 0.785861 

Epoch 4
-------------------------------
loss: 0.538653  [   64/21103]
loss: 0.281369  [ 6464/21103]
loss: 0.488981  [12864/21103]
loss: 0.573146  [19264/21103]
Test Error: 
 Accuracy: 71.4%, Avg loss: 0.814851 

Epoch 5
-------------------------------
loss: 0.512342  [   64/21103]
loss: 0.250270  [ 6464/21103]
loss: 0.525896  [12864/21103]
loss: 0.532487  [19264

In [29]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [281]:
classes = list(conversion.keys())

correct = 0
total = 0

model.eval()
for x, label_batch in test_dataloader:
    with torch.no_grad():
        x = x.to(device)
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[label_batch[0].item()]
        if predicted == actual:
            correct += 1
        total += 1

print(f"Accuracy: {correct/total*100}%")

Accuracy: 74.69879518072288%
